This notebook simply goes to a number of positions in workd space and reads out the joint configugarion of the robot at the position.
These data is used in `scene_utils` to workaround the limitation of a missing exposed inverse kinematics solver as well as the missing `beam_to_cart_pos` method.
The data is collected in a grid pattern in the workspace of the robot.

In [8]:
from pprint import pprint

import numpy as np
import glfw

from alr_sim.sims.SimFactory import SimRepository
from alr_sim.core import Scene

from alr_sim_tools.scene_utils import reset_scene

In [9]:
FACTORY_SRTING = "mj_beta"
DT=0.0002
RENDER_MODE = Scene.RenderMode.HUMAN
CAMERA_POSITION = (0.5, 0.0, 1.0)
CAMERA_QUAT = (-0.70710678, 0, 0, 0.70710678)

MOVEMENT_DURATION = 2
WAIT_DURATIONS_BETWEEN_MOVEMENTS = 0.5
TARGET_QUAT = (0, 1, 0, 0)

MIN_X = 0.0
MAX_X = 0.0
DELTA_X = 0.1
MIN_Y = 0.5
MAX_Y = 0.5
DELTA_Y = 0.1
MIN_Z = 0.1
MAX_Z = 0.6
DELTA_Z = 0.05

JOINT_VALUE_DECIMALS = 5

In [10]:
positions = np.meshgrid(
    np.arange(MIN_X, MAX_X + DELTA_X, DELTA_X),
    np.arange(MIN_Y, MAX_Y + DELTA_Y, DELTA_Y),
    np.arange(MIN_Z, MAX_Z + DELTA_Z, DELTA_Z),
)
positions = np.array(positions).T.reshape(-1, 3)

positions


array([[0.5 , 0.  , 0.1 ],
       [0.5 , 0.  , 0.15],
       [0.5 , 0.  , 0.2 ],
       [0.5 , 0.  , 0.25],
       [0.5 , 0.  , 0.3 ],
       [0.5 , 0.  , 0.35],
       [0.5 , 0.  , 0.4 ],
       [0.5 , 0.  , 0.45],
       [0.5 , 0.  , 0.5 ],
       [0.5 , 0.  , 0.55],
       [0.5 , 0.  , 0.6 ]])

In [11]:
sim_factory = SimRepository.get_factory(FACTORY_SRTING)
scene = sim_factory.create_scene(object_list=[], dt=DT, render=RENDER_MODE)
agent = sim_factory.create_robot(scene, dt=DT)

scene.start()
scene.start_logging()

In [12]:
world_2_joint_coords = {}

for pos in positions:
    pos = tuple(pos.tolist())
    print(f"Going to {pos}") 
    agent.gotoCartPositionAndQuat(pos, TARGET_QUAT, MOVEMENT_DURATION)
    agent.wait(WAIT_DURATIONS_BETWEEN_MOVEMENTS)
    world_2_joint_coords[pos] = tuple(agent.current_j_pos) 


Going to (0.5, 0.0, 0.1)
Going to (0.5, 0.0, 0.15000000000000002)
Going to (0.5, 0.0, 0.20000000000000004)
Going to (0.5, 0.0, 0.25000000000000006)
Going to (0.5, 0.0, 0.30000000000000004)
Going to (0.5, 0.0, 0.3500000000000001)
Going to (0.5, 0.0, 0.40000000000000013)
Going to (0.5, 0.0, 0.45000000000000007)
Going to (0.5, 0.0, 0.5000000000000001)
Going to (0.5, 0.0, 0.5500000000000002)
Going to (0.5, 0.0, 0.6000000000000002)


In [13]:
scene.stop_logging()
reset_scene(FACTORY_SRTING, scene, agent)
glfw.terminate()

In [14]:
world_2_joint_coords_ = {}

for pos, j_coords in world_2_joint_coords.items():
    world_2_joint_coords_[(tuple(pos), tuple(TARGET_QUAT))] = tuple(
        round(j, JOINT_VALUE_DECIMALS) for j in j_coords
    )

pprint(world_2_joint_coords_, width=160)

{((0.5, 0.0, 0.1), (0, 1, 0, 0)): (-0.0001, 0.26738, -0.00014, -2.39607, -0.00011, 2.66243, 0.78532),
 ((0.5, 0.0, 0.15000000000000002), (0, 1, 0, 0)): (1e-05, 0.15345, -1e-05, -2.39756, -2e-05, 2.55245, 0.78545),
 ((0.5, 0.0, 0.20000000000000004), (0, 1, 0, 0)): (1e-05, 0.04767, 1e-05, -2.37694, 3e-05, 2.42672, 0.78542),
 ((0.5, 0.0, 0.25000000000000006), (0, 1, 0, 0)): (1e-05, -0.04058, 0.0, -2.33564, 5e-05, 2.29731, 0.7854),
 ((0.5, 0.0, 0.30000000000000004), (0, 1, 0, 0)): (0.0, -0.10949, -0.0, -2.27468, 6e-05, 2.16741, 0.7854),
 ((0.5, 0.0, 0.3500000000000001), (0, 1, 0, 0)): (-0.0, -0.1585, -1e-05, -2.19482, 7e-05, 2.03845, 0.7854),
 ((0.5, 0.0, 0.40000000000000013), (0, 1, 0, 0)): (-0.0, -0.1877, -1e-05, -2.09644, 8e-05, 1.91072, 0.7854),
 ((0.5, 0.0, 0.45000000000000007), (0, 1, 0, 0)): (-1e-05, -0.19738, -2e-05, -1.97912, 8e-05, 1.78354, 0.7854),
 ((0.5, 0.0, 0.5000000000000001), (0, 1, 0, 0)): (-1e-05, -0.18743, -2e-05, -1.84116, 8e-05, 1.65532, 0.7854),
 ((0.5, 0.0, 0.550000